In [202]:
import polars as pl
from pprint import pprint

In [203]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list, type:any=pl.Int64):
    data = data.with_columns(pl.col(fix_list).str.split(","))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip_chars()))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().cast(pl.Int64, strict=False)))

    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.any()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})
    
    # data = data.with_columns(pl.col(fix_list).str.strip().str.replace("","0")).cast(conversion_dict)

    return data


# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df, new_position, col_name):
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder


In [204]:
def prep_csv(dataframe:pl.DataFrame):
    import numpy as np
    type_list = dataframe.dtypes
    mask = np.array([isinstance(column, pl.List) for column in type_list])
    column_list = np.array(dataframe.columns)
    list_columns = column_list[mask]
    print(list_columns)

    for column in list_columns:
        dataframe = dataframe.with_columns(pl.col(column).cast(pl.List(pl.Utf8)).list.join(","))
    return dataframe
    

In [205]:
rozvrh_by_kat = pl.read_csv("source_tables/rozvrhy_fakulta.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])
#print(rozvrh_by_kat.head())
predmety_by_kat = pl.read_csv("source_tables/predmety_fakulta.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])
print(predmety_by_kat.__len__())
predmety_by_kat.filter(pl.col("zkratka") == "P522").head()
#predmety_by_kat = pl.read_excel("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.xlsx", sheet_id=1, engine="openpyxl").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])

3083


katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""KFY""","""P522""",2023,"""Vybrané partie…","""Vybrané partie…","""A""","""A""",4,"""NE""","""""","""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Feynman R.P.,…","""""","""""","""""","""A""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je věnová…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""1. Termodynami…","""Výuka v anglič…","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""N""","""""",0,"""""","""Mgr.""","""A""",null
"""KBI""","""P522""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",7,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'http://biolog…","""""","""Monologická (v…","""Písemná zkoušk…","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: Ne ví…","""1. Vnitřní pro…","""Znalost biolog…","""""","""""","""""","""Čeština, Angli…","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""KCH""","""P522""",2023,"""Odborný seminá…","""Odborný seminá…","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Jan…","""'doc. Ing. Jan…",[1064],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Seminář má umo…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null


In [206]:
bez_problemu = predmety_by_kat.select(["katedra", "zkratka", "nazev"]).with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier"))

In [207]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [208]:
predmety_by_kat.filter(pl.col("zkratka") == "B317").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""KCH""","""B317""",2023,"""Membránové sep…","""Membránové sep…","""A""","""A""",3,"""NE""","""""","""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""'Ing. Zuzana P…","""'Ing. Zuzana P…",[3705],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""KCH/B104""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Prezentace vyb…","""1. Historie me…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0]


In [209]:
rozvrh_by_kat.head(5)
#rozvrh_by_kat.__len__()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
476309,"""Metody příprav…","""CNB""","""M101""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.26""","""12""",5,4,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""4""","""4""",1,"""10:00""","""10:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N ACH""","""UJEP""","""""","""6.10.2023""","""5.1.2024""","""R""","[332, 280, 308]","""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská, Malý, …",476309,"""""","""""","""PETRUSPRF""",""""""
476313,"""Vědecká komuni…","""CNB""","""M100""","""""","""280""",[280],"""Jan""","""Malý""","""Mgr.""","""Ph.D.""","""A""","""A""","""25""",2023,"""""","""""","""""",20,9,"""Přednáška""","""Př""","""A""","""""","""""","""""","""""","""""",1,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.BI N AB,1.BI…","""""","""""","""""","""""","""R""","[280, 278, … 3661]","""Mgr. Jan Malý,…","""'Mgr. Jan Malý…","""Malý, Hajer, J…",476313,"""""","""""","""PETRUSPRF""",""""""
476314,"""Vědecká komuni…","""CNB""","""M100""","""""","""280""",[280],"""Jan""","""Malý""","""Mgr.""","""Ph.D.""","""A""","""A""","""25""",2023,"""""","""""","""""",20,9,"""Cvičení""","""Cv""","""A""","""""","""""","""""","""""","""""",1,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.BI N AB,1.BI…","""""","""""","""""","""""","""R""","[280, 278, … 3661]","""Mgr. Jan Malý,…","""'Mgr. Jan Malý…","""Malý, Hajer, J…",476314,"""""","""""","""PETRUSPRF""",""""""
484586,"""Mikrofluidní s…","""CNB""","""MSBA""","""""","""1503""",[1503],"""Marcel""","""Štofik""","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""4.19""","""16""",8,1,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""2""","""2""",1,"""08:00""","""08:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.NANO,3.NANO""","""UJEP""","""""","""22.2.2024""","""16.5.2024""","""R""",[1503],"""Mgr. Marcel Št…","""'Mgr. Marcel Š…","""Štofik""",484586,"""""","""""","""RSMEJKAL""",""""""
489242,"""Fyziologie živ…","""CNB""","""KB502""","""""","""8024""",[8024],"""Stanislav""","""Vinopal""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""ZV""","""B2""","""12""",1,0,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""","""3""","""10""",8,"""09:00""","""16:50""",14,14,"""Jiný""","""J""","""""","""N""","""N""","""Automaticky vy…","""3.BI CHE K""","""UJEP""","""""","""6.4.2024""","""6.4.2024""","""R""","[8024, 5275]","""Ing. Stanislav…","""'Ing. Stanisla…","""Vinopal, Ruper…",489242,"""""","""""","""RSMEJKAL""",""""""


In [210]:
set(rozvrh_by_kat.columns) & set(predmety_by_kat.columns)

{'katedra', 'nazev', 'rok', 'zkratka'}

In [211]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,162,237,0,0,0,0,0,0,162,0,0,66,160,0,0,0,0,0,0,0,0,473,473,172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,237,0,0,0,0,0,0,0,0


In [212]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,351,0,0,1618,0,0,1262,0,0,2673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1901


In [213]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [214]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) 
predmety_s_akci.filter(pl.col("zkratka") == "P522").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""KBI""","""P522""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""N""",7,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""'Mgr. Marian R…","""'Mgr. Marian R…","[5275, 8024]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'http://biolog…","""""","""Monologická (v…","""Písemná zkoušk…","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: Ne ví…","""1. Vnitřní pro…","""Znalost biolog…","""""","""""","""""","""Čeština, Angli…","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""KCH""","""P522""",2023,"""Odborný seminá…","""Odborný seminá…","""A""","""A""",1,"""NE""","""""","""'doc. Ing. Jan…","""'doc. Ing. Jan…",[1064],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Seminář má umo…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null
"""KFY""","""P522""",2023,"""Vybrané partie…","""Vybrané partie…","""A""","""A""",4,"""NE""","""""","""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""'doc. RNDr. In…","""'doc. RNDr. In…",[606],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Feynman R.P.,…","""""","""""","""""","""A""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je věnová…","""Zkouška""","""ANO""","""Kombinovaná""","""""","""1. Termodynami…","""Výuka v anglič…","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""N""","""""",0,"""""","""Mgr.""","""A""",null


# Vyučující a garanti

In [215]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")
garant = pl.col("garanti")
cvicici = pl.col("cvicici")
prednasejici = pl.col("prednasejici")
seminarici = pl.col("seminarici")

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [216]:
zkratky = predmety_s_akci.filter(pl.col("garantiUcitIdno").is_null()).select(["katedra", "zkratka", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
zkratky.write_excel("results_xlsx/bez_garanta.xlsx")
zkratky.write_csv("results_csv/bez_garanta.csv")
zkratky
# print(list(zkratky), sep=", ")

bez_problemu = bez_problemu.join(zkratky.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")
bez_problemu.head()


166 Předmětů nemá zadaného garanta


katedra,zkratka,nazev,identifier
str,str,str,str
"""CNB""","""DNA""","""Příprava a fun…","""DNA/CNB"""
"""CNB""","""KPCIM""","""Prakt. cvičení…","""KPCIM/CNB"""
"""CNB""","""MPNM""","""Met. přípr. na…","""MPNM/CNB"""
"""CNB""","""NA08""","""Metody příprav…","""NA08/CNB"""
"""CNB""","""NA13""","""Praktika z cha…","""NA13/CNB"""


> Předměty podezřelé z více garantů:

In [217]:
# predmety_s_akci.filter( > pl.lit())
vice_garantu = predmety_s_akci.with_columns(pl.col("garantiUcitIdno").list.len().alias("pocet garantu")
    ).select(
        ["garantiUcitIdno", "garanti","katedra", "zkratka", "pocet garantu"]
    ).filter(pl.col("pocet garantu") > 1)
vice_garantu.write_excel("results_xlsx/vice_garantu.xlsx")
vice_garantu_csv = prep_csv(vice_garantu)
vice_garantu_csv.write_csv("results_csv/vice_garantu.csv")
vice_garantu.head(10)
bez_problemu = bez_problemu.join(vice_garantu.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

['garantiUcitIdno']


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [218]:
ma_prednasku = jednotek_prednasek != 0
nema_prednasejiciho = pl.col("prednasejiciUcitIdno").is_null()

chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho).select("katedra", "zkratka", "prednasejici", "prednasejiciUcitIdno", "jednotekPrednasek", "jednotkaPrednasky")
chybi_prednasejici.write_excel("results_xlsx/chybi_prednasejici.xlsx")

#prep_csv(chybi_prednasejici).write_csv("results/chybi_prednasejici.csv")
chybi_prednasejici.head()
#bez_problemu = bez_problemu.join(chybi_prednasejici.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

katedra,zkratka,prednasejici,prednasejiciUcitIdno,jednotekPrednasek,jednotkaPrednasky
str,str,str,list[i64],i64,str
"""KBI""","""0023""","""""",null,2,"""HOD/TYD"""
"""KPRF""","""0101""","""""",null,2,"""HOD/TYD"""
"""KGEO""","""0159""","""""",null,1,"""HOD/TYD"""
"""KGEO""","""B210""","""""",null,2,"""HOD/TYD"""
"""KGEO""","""B414""","""""",null,2,"""HOD/TYD"""


In [219]:
test = prep_csv(chybi_prednasejici)
test.write_csv("results_csv/chybi_prednasejici.csv")

['prednasejiciUcitIdno']


In [220]:
ma_cviceni = jednotek_cviceni != 0
nema_cviciciho = pl.col("cviciciUcitIdno").is_null()

chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho).select("katedra", "zkratka", "cvicici", "cviciciUcitIdno", "jednotekCviceni", "jednotkaCviceni")
chybi_cvicici.write_excel("results_xlsx/chybi_cvicici.xlsx")
chybi_cvicici_csv = prep_csv(chybi_cvicici)
chybi_cvicici_csv.write_csv("results_csv/chybi_cvicici.csv")
chybi_cvicici.head()
bez_problemu = bez_problemu.join(chybi_cvicici.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

['cviciciUcitIdno']


In [221]:
ma_seminare = jednotek_seminare != 0
nema_seminariciho = pl.col("seminariciUcitIdno").is_null()

chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho).select("katedra", "zkratka", "seminarici", "seminariciUcitIdno", "jednotekSeminare", "jednotkaSeminare")
chybi_seminarici.write_excel("results_xlsx/chybi_seminarici.xlsx")
chybi_seminarici_csv = prep_csv(chybi_seminarici)
chybi_seminarici_csv.write_csv("results_csv/chybi_seminarici.xlsx")
bez_problemu = bez_problemu.join(chybi_seminarici.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

['seminariciUcitIdno']


> Zde jsou předměty, ve kterých učitel vede seminář:

In [222]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

In [223]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

32 předmětů nemá přednášejícího
88 předmětů nemá cvičícího
37 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [224]:
filter_query = (garant != cvicici) & (garant != seminarici) & (garant != prednasejici) & (ma_prednasku | ma_cviceni | ma_seminare)
predmety_kde_garant_neuci = predmety_s_akci.filter(filter_query).select("katedra", "zkratka", "garanti", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno", "cvicici", "cviciciUcitIdno", "seminarici", "seminariciUcitIdno")
predmety_kde_garant_neuci.write_excel("results_xlsx/predmety_kde_garant_neuci.xlsx")
predmety_kde_garant_neuci_csv = prep_csv(predmety_kde_garant_neuci)
predmety_kde_garant_neuci_csv.write_csv("results_csv/predmety_kde_garant_neuci.csv")
predmety_kde_garant_neuci.__len__()
bez_problemu = bez_problemu.join(predmety_kde_garant_neuci.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

['garantiUcitIdno' 'prednasejiciUcitIdno' 'cviciciUcitIdno'
 'seminariciUcitIdno']


> v těchto předmětech je přednáška, ale nedělá jí garant:

In [225]:
garant_neni_prednasejici = (jednotek_prednasek != 0) & (garant != prednasejici)
garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici).select("katedra", "zkratka", "garanti", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno")
garant_neprednasi.write_excel("results_xlsx/garant_neprednasi.xlsx")
garant_neprednasi_csv = prep_csv(garant_neprednasi)
garant_neprednasi_csv.write_csv("results_csv/garant_neprednasi.csv")
garant_neprednasi.__len__()
bez_problemu = bez_problemu.join(garant_neprednasi.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

['garantiUcitIdno' 'prednasejiciUcitIdno']


## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [226]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
476309,"""Metody příprav…","""CNB""","""M101""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.26""","""12""",5,4,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""4""","""4""",1,"""10:00""","""10:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N ACH""","""UJEP""","""""","""6.10.2023""","""5.1.2024""","""R""","[332, 280, 308]","""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská, Malý, …",476309,"""""","""""","""PETRUSPRF""",""""""
476313,"""Vědecká komuni…","""CNB""","""M100""","""""","""280""",[280],"""Jan""","""Malý""","""Mgr.""","""Ph.D.""","""A""","""A""","""25""",2023,"""""","""""","""""",20,9,"""Přednáška""","""Př""","""A""","""""","""""","""""","""""","""""",1,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.BI N AB,1.BI…","""""","""""","""""","""""","""R""","[280, 278, … 3661]","""Mgr. Jan Malý,…","""'Mgr. Jan Malý…","""Malý, Hajer, J…",476313,"""""","""""","""PETRUSPRF""",""""""
476314,"""Vědecká komuni…","""CNB""","""M100""","""""","""280""",[280],"""Jan""","""Malý""","""Mgr.""","""Ph.D.""","""A""","""A""","""25""",2023,"""""","""""","""""",20,9,"""Cvičení""","""Cv""","""A""","""""","""""","""""","""""","""""",1,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.BI N AB,1.BI…","""""","""""","""""","""""","""R""","[280, 278, … 3661]","""Mgr. Jan Malý,…","""'Mgr. Jan Malý…","""Malý, Hajer, J…",476314,"""""","""""","""PETRUSPRF""",""""""
484586,"""Mikrofluidní s…","""CNB""","""MSBA""","""""","""1503""",[1503],"""Marcel""","""Štofik""","""Mgr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""4.19""","""16""",8,1,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""2""","""2""",1,"""08:00""","""08:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.NANO,3.NANO""","""UJEP""","""""","""22.2.2024""","""16.5.2024""","""R""",[1503],"""Mgr. Marcel Št…","""'Mgr. Marcel Š…","""Štofik""",484586,"""""","""""","""RSMEJKAL""",""""""
489242,"""Fyziologie živ…","""CNB""","""KB502""","""""","""8024""",[8024],"""Stanislav""","""Vinopal""","""Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""ZV""","""B2""","""12""",1,0,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""","""3""","""10""",8,"""09:00""","""16:50""",14,14,"""Jiný""","""J""","""""","""N""","""N""","""Automaticky vy…","""3.BI CHE K""","""UJEP""","""""","""6.4.2024""","""6.4.2024""","""R""","[8024, 5275]","""Ing. Stanislav…","""'Ing. Stanisla…","""Vinopal, Ruper…",489242,"""""","""""","""RSMEJKAL""",""""""


In [227]:
# PROZATÍMNÍ NASTAVENÍ FILTRUJE AKCE BEZ DATUMU

maly_rozvrh = rozvrh_by_kat.select(["katedra","zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok", "datumOd", "datumDo", "hodinaSkutOd", "hodinaSkutDo"])
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"}).filter(pl.col("datumOd").str.len_chars() > 0)
maly_rozvrh.filter(pl.col("katedra").is_null()).head()

katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo
str,str,i64,str,i64,str,str,str,str


In [228]:
# predmety_by_kat

In [229]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""CNB""","""DNA""",2023,"""Příprava a fun…","""Příprava a fun…","""N""","""A""",2,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""""","""""",null,"""'Ing. Stanisla…","""'Ing. Stanisla…",[8024],"""""","""""",null,"""""","""""","""""","""""","""KBI/BP102, KCH…","""""","""""","""'ALBERTS, Bruc…","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""DNU/SEM""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0]
"""CNB""","""KPCIM""",2023,"""Prakt. cvičení…","""Praktická cvič…","""A""","""N""",2,"""NE""","""""","""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'B. Králová, L…","""""","""""","""""","""A""",0,"""HOD/SEM""",12,"""HOD/SEM""",0,"""HOD/SEM""","""""","""Zápočet""","""NE""","""Kombinovaná""","""Nutnou podmínk…","""Imunoanalytick…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""Praktická cvič…","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0]
"""CNB""","""MPNM""",2023,"""Met. přípr. na…","""Metody příprav…","""N""","""A""",3,"""NE""","""""","""'prof. Ing. Zd…","""'prof. Ing. Zd…",[332],"""'prof. Ing. Zd…","""'prof. Ing. Zd…",[332],"""'prof. Ing. Zd…","""'prof. Ing. Zd…",[332],"""""","""""",null,"""""","""""","""""","""""","""CNB/UNM, KCH/A…","""""","""""","""'R. Prucek. Př…","""KFY/MPNM""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zkouška""","""ANO""","""Kombinovaná""","""Účast na předn…","""1. chemické me…","""""","""""","""""","""""","""""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0]
"""CNB""","""NA08""",2023,"""Metody příprav…","""Metody příprav…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Ja…","""'doc. RNDr. Ja…",[308],"""'prof. Ing. Zd…","""'prof. Ing. Zd…","[332, 280, 308]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KCH/NA03""","""'Challa S. S. …","""KFY/NA08""","""""","""Písemná zkoušk…","""N""",1,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""Předmět bude z…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",null
"""CNB""","""NA13""",2023,"""Praktika z cha…","""Praktika z cha…","""N""","""A""",3,"""NE""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…",[2384],"""""","""""",null,"""'prof. Ing. Z

In [230]:
ciselnik_ucitelu = pl.read_csv("source_tables/ciselnik_ucitelu.csv").select("nazev", "key").rename({"nazev":"jmena", "key":"idno"})
ciselnik_ucitelu.head()

jmena,idno
str,i64
"""Abrahamová Pet…",8409
"""Ač Alexander, …",9316
"""Adamec Slavomí…",5432
"""Adámek Pavel, …",7443
"""Adam Julie, M.…",9249


In [231]:
male_predmety = predmety_by_kat.with_columns(pl.concat_str(pl.col("zkratka"), pl.col("katedra")).alias("zkratka + katedra")).select(pl.col(["zkratka", "katedra", "zkratka + katedra", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"])).unique(subset="zkratka + katedra").drop("zkratka + katedra")
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.filter(pl.col("zkratka") == "B317").head()

zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejici,prednasejiciUcitIdno,cvicici,cviciciUcitIdno,seminarici,seminariciUcitIdno
str,str,i64,str,list[i64],str,list[i64],str,list[i64],str,list[i64]
"""B317""","""KCH""",2023,"""Membránové sep…",[3705],"""'Ing. Zuzana P…",[3705],"""""",null,"""""",null


In [232]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno")
prednasejici_jmena = male_predmety.select("prednasejici").rename({"prednasejici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
prednasejici_jmena = prednasejici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_prednasejici = filtrovani_prednasejici.with_columns(prednasejici_jmena).filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    pl.col("prednasejiciUcitIdno").alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_excel("results_xlsx/prednasejici_bez_prednasek.xlsx")
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_csv("results_csv/prednasejici_bez_prednasek.csv")
bez_problemu = bez_problemu.join(prednasejici_bez_prednasek.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

In [233]:
#WEIRD STUFF: Učitel 8027 u KGEO/B508
# otherwise looks pretty good

male_prednasky = maly_rozvrh.filter(pl.col("typAkceZkr") == "Př")
joined_prednasky = male_prednasky.join(male_predmety.select("zkratka", "katedra", "nazev", "prednasejici", "prednasejiciUcitIdno"), "zkratka", "left").unique()
prednasky_bez_prednasejicich = joined_prednasky.filter(pl.col("idno").is_in(pl.col("prednasejiciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

prednasky_bez_prednasejicich = prednasky_bez_prednasejicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("prednasejiciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

prednasky_bez_prednasejicich.write_csv("results_csv/prednasky_bez_prednasejicich.csv")
prednasky_bez_prednasejicich.write_excel("results_xlsx/prednasky_bez_prednasejicich.xlsx")

bez_problemu = bez_problemu.join(prednasky_bez_prednasejicich.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

print(prednasky_bez_prednasejicich.__len__())
prednasky_bez_prednasejicich.head(15)

115


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,prednasejici,prednasejiciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KPRF""","""0101""",556,"""Př""",2023,"""2.10.2023""","""18.12.2023""","""16:00""","""17:50""","""KPRF""","""Úvod do meteor…","""""",null,null
"""KCH""","""B100""",2856,"""Př""",2023,"""3.10.2023""","""2.1.2024""","""12:00""","""13:50""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Škvorová Magda…"
"""KCH""","""B104""",8421,"""Př""",2023,"""20.2.2024""","""14.5.2024""","""12:00""","""13:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Vojtíšek Pavel…"
"""KGEO""","""B203""",3164,"""Př""",2023,"""21.2.2024""","""15.5.2024""","""17:00""","""18:50""","""KGEO""","""Geografie Česk…","""'doc. Mgr. Pav…","""1434""","""Riezner Jiří, …"
"""KGEO""","""B206""",6384,"""Př""",2023,"""21.2.2024""","""15.5.2024""","""08:00""","""09:50""","""KGEO""","""Regionální geo…","""'doc. RNDr. Ji…","""312""","""Trahorsch Petr…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KGEO""","""B508""",8027,"""Př""",2023,"""3.10.2023""","""2.1.2024""","""17:00""","""18:50""","""KGEO""","""Ochrana přírod…","""'RNDr. Tomáš M…","""1472""",null
"""KGEO""","""B602""",3528,"""Př""",2023,"""19.2.2024""","""13.5.2024""","""14:00""","""15:50""","""KGEO""","""Specializační …","""'Mgr. Petr Mey…","""5855""","""Kučerová Silvi…"
"""KGEO""","""B602""",9521,"""Př""",2023,"""19.2.2024""","""13.5.2024""","""14:00""","""15:50""","""KGEO""","""Specializační …","""'Mgr. Petr Mey…","""5855""","""Vaverka Lukáš,…"


In [234]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno")
cvicici_jmena = male_predmety.select("cvicici").rename({"cvicici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
cvicici_jmena = cvicici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_cvicici = filtrovani_cvicici.with_columns(cvicici_jmena).filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    pl.col("cviciciUcitIdno").alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results_xlsx/cvicici_bez_cviceni.xlsx")
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno", "jmena").sort("cviciciUcitIdno").write_csv("results_csv/cvicici_bez_cviceni.csv")

bez_problemu = bez_problemu.join(cvicici_bez_cviceni.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

In [235]:
#WEIRD STUFF: KGEO/0165 má napsaná cvičení (jak v rozvrhu tak předmětu) ale nemá napsanýho žádnýho učitele (fakt ani garanta, ani jednoho přednášejícího/cvičícího/etc.), tf?

male_cviceni = maly_rozvrh.filter(pl.col("typAkceZkr") == "Cv")
joined_cviceni = male_cviceni.join(male_predmety.select("zkratka", "katedra", "nazev", "cvicici", "cviciciUcitIdno"), "zkratka", "left").unique()
cviceni_bez_cvicich = joined_cviceni.filter(pl.col("idno").is_in(pl.col("cviciciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

cviceni_bez_cvicich = cviceni_bez_cvicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("cviciciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

cviceni_bez_cvicich.write_csv("results_csv/cviceni_bez_cvicich.csv")
cviceni_bez_cvicich.write_excel("results_xlsx/cviceni_bez_cvicich.xlsx")

bez_problemu = bez_problemu.join(cviceni_bez_cvicich.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

print(cviceni_bez_cvicich.__len__())
cviceni_bez_cvicich.head(15)

260


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,cvicici,cviciciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KGEO""","""0165""",9282,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""12:00""","""13:50""","""KGEO""","""Hodnotová orie…","""""",null,"""Vyhnánková Kat…"
"""KFY""","""0219""",7723,"""Cv""",2023,"""4.10.2023""","""3.1.2024""","""16:00""","""17:50""","""KFY""","""CFD Simulace""","""'doc. Ing. Jar…","""1803""","""Kramoliš David…"
"""KGEO""","""0A00""",6919,"""Cv""",2023,"""4.10.2023""","""3.1.2024""","""13:00""","""14:50""","""KGEO""","""Diplomová prác…","""'RNDr. Silvie …","""3528""","""Píša Jan, Mgr.…"
"""KCH""","""ACH""",9301,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""11:00""","""11:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Greguš Viktor,…"
"""KCH""","""B100""",9049,"""Cv""",2023,"""6.10.2023""","""15.12.2023""","""14:00""","""17:50""","""KCH""","""Úvod do chemie…","""'doc. RNDr. Ja…","""2855""","""Dědičová Šárka…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KCH""","""B104""",9301,"""Cv""",2023,"""21.2.2024""","""15.5.2024""","""11:00""","""11:50""","""KCH""","""Anorganická ch…","""'RNDr. Václav …","""4310""","""Greguš Viktor,…"
"""KGEO""","""B104""",8819,"""Cv""",2023,"""5.10.2023""","""4.1.2024""","""10:00""","""11:50""","""KGEO""","""Geografická ka…","""'doc. PhDr. RN…","""3452, 5855""","""Bartůněk Marti…"
"""KCH""","""B107""",4117,"""Cv""",2023,"""2.5.2024""","""2.5.2024""","""13:00""","""13:50""","""KCH""","""Prezentace kat…","""'doc. Ing. Jan…","""1064""","""Vrtoch Ľuboš, …"


In [236]:
# seminařicí nemá seminář:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno")
seminarici_jmena = male_predmety.select("seminarici").rename({"seminarici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
seminarici_jmena = seminarici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_seminarici = filtrovani_seminarici.with_columns(seminarici_jmena).filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    pl.col("seminariciUcitIdno").alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_csv("results_csv/seminarici_bez_seminare.csv")
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results_xlsx/seminarici_bez_seminare.xlsx")

bez_problemu = bez_problemu.join(seminarici_bez_seminare.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")

In [237]:
male_seminare = maly_rozvrh.filter(pl.col("typAkceZkr") == "Se")
joined_seminare = male_seminare.join(male_predmety.select("zkratka", "katedra", "nazev", "seminarici", "seminariciUcitIdno"), "zkratka", "left").unique()
seminare_bez_seminaricich = joined_seminare.filter(pl.col("idno").is_in(pl.col("seminariciUcitIdno")).not_() & ((pl.col("katedra") == pl.col("katedra_right")) | pl.col("katedra_right").is_null()))

seminare_bez_seminaricich = seminare_bez_seminaricich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("seminariciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

seminare_bez_seminaricich.write_csv("results_csv/seminare_bez_seminaricich.csv")
seminare_bez_seminaricich.write_excel("results_xlsx/seminare_bez_seminaricich.xlsx")

bez_problemu = bez_problemu.join(seminare_bez_seminaricich.with_columns(pl.concat_str([pl.col("zkratka"), pl.col("katedra")], separator="/").alias("identifier")),"identifier" ,"anti")
print(seminare_bez_seminaricich.__len__())
seminare_bez_seminaricich.head(6)

67


katedra,zkratka,idno,typAkceZkr,rok,datumOd,datumDo,hodinaSkutOd,hodinaSkutDo,katedra_right,nazev,seminarici,seminariciUcitIdno,jmena
str,str,i64,str,i64,str,str,str,str,str,str,str,str,str
"""KI""","""0204""",null,"""Se""",2023,"""20.2.2024""","""14.5.2024""","""15:00""","""15:50""","""KI""","""Microsoft SQL …","""'Mgr. Květuše …","""835""",null
"""KI""","""0204""",null,"""Se""",2023,"""27.2.2024""","""7.5.2024""","""15:00""","""16:50""","""KI""","""Microsoft SQL …","""'Mgr. Květuše …","""835""",null
"""KCH""","""B213""",8946,"""Se""",2023,"""20.2.2024""","""14.5.2024""","""10:00""","""11:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK213""",8946,"""Se""",2023,"""20.4.2024""","""20.4.2024""","""13:00""","""16:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK213""",8946,"""Se""",2023,"""6.4.2024""","""6.4.2024""","""13:00""","""16:50""","""KCH""","""Odborná anglič…","""'doc. Ing. Jan…","""1064""","""Paříková Anna,…"
"""KCH""","""BK214""",4226,"""Se""",2023,"""10.5.2024""","""10.5.2024""","""09:00""","""12:50""","""KCH""","""Odborný seminá…","""'doc. Ing. Jan…","""1064""","""Kadlečková Iva…"


In [238]:
predmety_by_kat.filter(pl.col("zkratka") == "KTZI2").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64]
"""KI""","""KTZI2""",2023,"""Teoretické zák…","""Teoretické zák…","""N""","""A""",3,"""NE""","""""","""'RNDr. Martin …","""'RNDr. Martin …",[264],"""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""""","""""",null,"""""","""""","""""","""""","""KI/KTZI1""","""""","""""","""'Demel, J. Gra…","""""","""""","""""","""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je koncip…","""Zápočet""","""NE""","""Kombinovaná""","""Požadavky k zá…","""1. Rekurence a…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0]


#### Vypsání ne-problémů

In [239]:
# bez_problemu = bez_problemu.drop("identifier")
bez_problemu.write_csv("results_csv/bez_problemu.csv")
bez_problemu.write_excel("results_xlsx/bez_problemu.xlsx")
print(bez_problemu.__len__())
bez_problemu.head()

2200


katedra,zkratka,nazev
str,str,str
"""CNB""","""DNA""","""Příprava a fun…"
"""CNB""","""KPCIM""","""Prakt. cvičení…"
"""CNB""","""MPNM""","""Met. přípr. na…"
"""KBI""","""BK202""","""Základy parazi…"
"""KBI""","""BK220""","""Úvod do přípr.…"


### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`


In [240]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (3,)
 Series: 'typAkceZkr' [str]
 [
 	"Se"
 	"Cv"
 	"Př"
 ]]

### garant předmětu by měl být někde mezi vyučujícími rozvrhových akcí a pokud má předmět přednášku, měl by být i mezi přednášejícími v rámci rozvrhových akcí

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?